In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ModelWrapper import ModelWrapper, backward, gradient_descent_step, forward, compute_loss
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
model = ModelWrapper(net) # our prediction model

In [3]:
class fakeDataset(Dataset):
    def __init__(self, length, seed=0):
        self.data = torch.rand(length, 20)
        self.seed = seed

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.rand(1)

In [4]:
# data loaders that generate batches for our stochastic gradient descent
training_data = fakeDataset(10000)
valid_data = fakeDataset(100)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=64, shuffle=False)

In [5]:
learning_rate = 1e-3 # TODO

In [6]:
num_epochs = 100
all_losses = []
for epoch in tqdm(range(num_epochs)):  # loop over the dataset multiple times
    # train
    for data in train_dataloader:
        # get the inputs; data is a list of [inputs, labels]
        input, label = data

        prediction = forward(model, input)            # TODO: Use our current model to predict
        loss = compute_loss(prediction, label)        # TODO: Compute the loss between our prediction and the ground truth
        gradient = backward(model, input, label, loss)                       # TODO: Compute gradients for our model parameters
        model = gradient_descent_step(model, gradient, learning_rate)                # TODO: Update our model parameters with the computed gradients
        
    # evaluate
    validation_losses = []
    for data in valid_dataloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        with torch.no_grad():
            outputs = forward(model, inputs)        # TODO: Use our current model to predict
            loss = compute_loss(outputs, labels)    # TODO: Compute the loss between our prediction and the ground truth

            validation_losses.append(loss.item())
    print(f"epoch {epoch} validation loss {sum(validation_losses)}")
    all_losses.append(sum(validation_losses))
print('Finished Training')

epoch 0 validation loss 0.3042736351490021
epoch 1 validation loss 0.24123067408800125
epoch 2 validation loss 0.20428873598575592
epoch 3 validation loss 0.1966795176267624
epoch 4 validation loss 0.18778959661722183
epoch 5 validation loss 0.1432029753923416
epoch 6 validation loss 0.1496509239077568
epoch 7 validation loss 0.1767621636390686
epoch 8 validation loss 0.2049512416124344
epoch 9 validation loss 0.18714945018291473
epoch 10 validation loss 0.13981636613607407
epoch 11 validation loss 0.16237181425094604
epoch 12 validation loss 0.18227747827768326
epoch 13 validation loss 0.16066179424524307
epoch 14 validation loss 0.16325021535158157
epoch 15 validation loss 0.1522814705967903
epoch 16 validation loss 0.16813180595636368
epoch 17 validation loss 0.14612016081809998
epoch 18 validation loss 0.16658736765384674
epoch 19 validation loss 0.1450727880001068
epoch 20 validation loss 0.15182381868362427
epoch 21 validation loss 0.15571466833353043
epoch 22 validation loss 0.2